In [65]:
import pandas as pd
import numpy as np

In [15]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')

In [37]:
states = [s.replace('[E]', '').replace(' ', '-') for s in list(
    dfs[0]['Flag, name andpostal abbreviation[12]']['Flag, name andpostal abbreviation[12]']
)]
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New-Hampshire',
 'New-Jersey',
 'New-Mexico',
 'New-York',
 'North-Carolina',
 'North-Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode-Island',
 'South-Carolina',
 'South-Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West-Virginia',
 'Wisconsin',
 'Wyoming']

In [43]:
def load_state(state):
    print(f'loading data for state {state}')
    page = pd.read_html(f'https://www.nytimes.com/interactive/2020/11/03/us/elections/results-{state}.html')
    df = page[1]
    df = pd.DataFrame(df.iloc[:-1]) # last row is 'view results'
    df['Total votes'] = df['Total votes'].astype(int)
    df['State'] = state
    return df.sort_values('Total votes', ascending = False)

In [44]:
result = {}
for s in states:
    s = s.lower()
    result[s] = load_state(s)

loading data for state alabama
loading data for state alaska
loading data for state arizona
loading data for state arkansas
loading data for state california
loading data for state colorado
loading data for state connecticut
loading data for state delaware
loading data for state florida
loading data for state georgia
loading data for state hawaii
loading data for state idaho
loading data for state illinois
loading data for state indiana
loading data for state iowa
loading data for state kansas
loading data for state kentucky
loading data for state louisiana
loading data for state maine
loading data for state maryland
loading data for state massachusetts
loading data for state michigan
loading data for state minnesota
loading data for state mississippi
loading data for state missouri
loading data for state montana
loading data for state nebraska
loading data for state nevada
loading data for state new-hampshire
loading data for state new-jersey
loading data for state new-mexico
loading 

In [61]:
us_df = pd.concat(result.values())
us_df.head()

,County,Margin,2016 margin,Est. votes reported,Est. votes reported.1,Total votes,Absentee,State,District,Town,Parish
4,Jefferson,Biden +13,D+7.3,>98%,NaN,324675,—,alabama,NaN,NaN,NaN
5,Madison,Trump +8,R+16.3,>98%,NaN,193955,—,alabama,NaN,NaN,NaN
0,Mobile,Trump +12,R+13.3,93%,NaN,181783,—,alabama,NaN,NaN,NaN
6,Shelby,Trump +41,R+49.4,>98%,NaN,114439,—,alabama,NaN,NaN,NaN
7,Baldwin,Trump +54,R+57.2,>98%,NaN,108945,—,alabama,NaN,NaN,NaN


In [62]:
# assuming other parties are very minimal and won't affect the votes estimation
us_df['2020_Who'] = np.where(us_df['Margin'].str.split('+', expand=True)[0].str.strip() == 'Biden','D','R')
us_df['2020_Margin'] = pd.to_numeric(us_df['Margin'].str.split('+', expand=True)[1])
us_df['2016_Who'] = us_df['2016 margin'].str.split('+', expand=True)[0].str.strip()
us_df['2016_Margin'] = pd.to_numeric(us_df['2016 margin'].str.split('+', expand=True)[1])
us_df.head()

,County,Margin,2016 margin,Est. votes reported,Est. votes reported.1,Total votes,Absentee,State,District,Town,Parish,2020_Who,2020_Margin,2016_Who,2016_Margin
4,Jefferson,Biden +13,D+7.3,>98%,NaN,324675,—,alabama,NaN,NaN,NaN,D,13.0,D,7.3
5,Madison,Trump +8,R+16.3,>98%,NaN,193955,—,alabama,NaN,NaN,NaN,R,8.0,R,16.3
0,Mobile,Trump +12,R+13.3,93%,NaN,181783,—,alabama,NaN,NaN,NaN,R,12.0,R,13.3
6,Shelby,Trump +41,R+49.4,>98%,NaN,114439,—,alabama,NaN,NaN,NaN,R,41.0,R,49.4
7,Baldwin,Trump +54,R+57.2,>98%,NaN,108945,—,alabama,NaN,NaN,NaN,R,54.0,R,57.2


In [64]:
us_df[['2020_Margin', '2016_Margin']].corr()

,2020_Margin,2016_Margin
2020_Margin,1.000000,0.969486
2016_Margin,0.969486,1.000000
